# IBTrACS Valid Training Dates

So far, we've seen how to pull GOES imagery and how to overlay IBTrACS data on the imagery. We'll use these same ideas now to obtain dates to fetch training data. Some things we'll need to consider are:

* Some bands aren't visible at night, so we don't want to fetch images that aren't useful.
    * To address this, we can start by using only infrared bands on the imager that capture clouds at night. If more data is needed, we can come back to this point and identify daytime visible bands.
* The geostationary view of the imager has a limited extent, so some IBTrACS data points may not be visible to the imager.
    * We can use the projection data of the GOES image filter out the latitudes and longitudes of the IBTrACS data.
* GOES data is available as of 10 April 2017, so we'll only be able to use a subset of the IBTrACS data.
    * We just need to make sure we exclude any IBTrACS data outside of the GOES imaging window.
* IBTrACS data is every 3 hours, while GOES is every 5 minutes.
    * We can assume that if a storm had a track observation at time $t$ and again at time $t$+3 hours, an image within that time has a storm in it.

Each of these points will play a role in how we automatically assess if an image is a positive or negative training sample.

Downloading the data will take some time, so first, I just want to build a list of dates that, according to the IBTrACS data, correspond to a time when a tropical cyclone was present ($y=1$) or was not present ($y=0$). At first, I think the goal should be to get an even number of positive and negative training samples, so I think it might be best to randomly pick a time for a positive example, then ensure that the next selection is a negative example, and so on.

In [1]:
# Import modules
import datetime
import numpy as np
from pyproj import Proj

# Import functions I've written
import sys
sys.path.append("../scripts")
import goes
import ibtracs

## Read IBTrACS Data

Now we read in the IBTrACS data from 2017 until now. Let's get rid of anything else before 10 April.

In [2]:
# Download or set the file path for the IBTrACS data
file_path = ibtracs.download_data(basin="ALL",data_dir='../data/ibtracs',overwrite=False)

# Read the data from the CSV
df = ibtracs.read_data(file_path)

# Restrict the data to the appropriate timespan of the GOES data
df = df.loc[df['ISO_TIME']>=datetime.datetime(2017,4,10)].copy(True)
df.tail()

,SID,SEASON,NUMBER,NAME,ISO_TIME,NATURE,LAT,LON,WMO_WIND,WMO_PRES,TRACK_TYPE,DIST2LAND,LANDFALL,IFLAG,STORM_SPEED,STORM_DIR
722157,2025333N05105,2025,104,UNNAMED,2025-11-29 12:00:00,NR,5.2,107.5,,,PROVISIONAL,140,135,O______________,10,65
722158,2025333N05105,2025,104,UNNAMED,2025-11-29 15:00:00,NR,5.4,108.0,,,PROVISIONAL,146,145,P______________,11,70
722159,2025333N05105,2025,104,UNNAMED,2025-11-29 18:00:00,NR,5.6,108.5,,,PROVISIONAL,169,169,O______________,10,55
722160,2025333N05105,2025,104,UNNAMED,2025-11-29 21:00:00,NR,6.0,108.8,,,PROVISIONAL,220,220,P______________,10,35
722161,2025333N05105,2025,104,UNNAMED,2025-11-30 00:00:00,NR,6.4,109.1,,,PROVISIONAL,272,,O______________,10,30


## Read GOES16 Image

Next, we'll read in the GOES16 image corresponding to 30 days ago. The image itself doesn't matter yet. Since the imager is geostationary, we can just take any image and use it to make sure our IBTrACS data all falls within the GOES16 full disc array.

In [3]:
# Set the parameters to download data
date = datetime.datetime.now()-datetime.timedelta(days=30)
position = 'goes-east'
product = 'ABI-L1b-RadF'
band = 13

# Get the GOES data
ds = goes.get_goes_file(position='goes-east', date=date, band=band, product=product)
display(ds)

<xarray.Dataset> Size: 235MB
Dimensions:                                           (y: 5424, x: 5424,
                                                       number_of_time_bounds: 2,
                                                       number_of_image_bounds: 2,
                                                       number_of_harmonization_coefficients: 3,
                                                       num_star_looks: 24,
                                                       band: 1)
Coordinates:
  * y                                                 (y) float32 22kB 0.1518...
  * x                                                 (x) float32 22kB -0.151...
    t                                                 datetime64[ns] 8B 2025-...
    y_image                                           float32 4B 0.0
    x_image                                           float32 4B 0.0
    band_id                                           (band) int8 1B 13
    band_wavelength                                   (band) float32 4B 10.33
    t_star_look                                       (num_star_looks) datetime64[ns] 192B ...
    band_wavelength_star_look                         (num_star_looks) float32 96B ...
Dimensions without coordinates: number_of_time_bounds, number_of_image_bounds,
                                number_of_harmonization_coefficients,
                                num_star_looks, band
Data variables: (12/39)
    Rad                                               (y, x) float32 118MB na...
    DQF                                               (y, x) float32 118MB na...
    time_bounds                                       (number_of_time_bounds) datetime64[ns] 16B ...
    goes_imager_projection                            int32 4B -2147483647
    y_image_bounds                                    (number_of_image_bounds) float32 8B ...
    x_image_bounds                                    (number_of_image_bounds) float32 8B ...
    ...                                                ...
    algorithm_dynamic_input_data_container            int32 4B -2147483647
    processing_parm_version_container                 int32 4B -2147483647
    algorithm_product_version_container               int32 4B -2147483647
    star_id                                           (num_star_looks) float32 96B ...
    channel_integration_time                          float64 8B nan
    channel_gain_field                                float64 8B nan
Attributes: (12/30)
    naming_authority:          gov.nesdis.noaa
    Conventions:               CF-1.7
    standard_name_vocabulary:  CF Standard Name Table (v35, 20 July 2016)
    institution:               DOC/NOAA/NESDIS > U.S. Department of Commerce,...
    project:                   GOES
    production_site:           WCDAS
    ...                        ...
    timeline_id:               ABI Mode 6
    date_created:              2025-11-17T22:09:58.3Z
    time_coverage_start:       2025-11-17T22:00:20.8Z
    time_coverage_end:         2025-11-17T22:09:52.8Z
    LUT_Filenames:             SpaceLookParams(FM4A_CDRL79RevA_DO_14_00_00)-7...
    id:                        c1393013-98fe-47f6-859e-62a6dd135aa8

## Handle Projections

In order to get the IBTrACS data relative to the GOES imagery, we need to get the projection of the GOES data.

In [4]:
# Get dataset projection data
sat_height = ds.goes_imager_projection.perspective_point_height
sat_lon = ds.goes_imager_projection.longitude_of_projection_origin
sat_sweep = ds.goes_imager_projection.sweep_angle_axis

# The projection x and y coordinates equals the scanning angle (in radians) multiplied by the satellite height
x = ds.variables['x'][:] * sat_height
y = ds.variables['y'][:] * sat_height

# Create X and Y meshgrids
X, Y = np.meshgrid(x, y)

# Create a pyproj geostationary map object
p = Proj(proj='geos', h=sat_height, lon_0=sat_lon, sweep=sat_sweep)

# Get latitudes and longitudes
lons, lats = p(X, Y, inverse=True)

## Filter Out with Bounding Box

A bounding box corresponding to the minimum and maximum latitude and longitudes covers more space than the full-disc, but the only way to really check is to loop through *all* of the IBTrACS data, project it onto the GOES projection, and then see if it's in the image. That will take a lot of time, where this will not, so let's use a bounding box as a first pass here to avoid unnecessary loopling.

In [5]:
# Get a simple bounding box based on min/max lat/lons
lons = np.where(lons==1e+30,np.nan,lons)
lats = np.where(lats==1e+30,np.nan,lats)
min_lat = np.nanmin(lats[lats != -np.inf])
max_lat = np.nanmax(lats[lats != np.inf])
min_lon = np.nanmin(lons[lons != -np.inf])
max_lon = np.nanmax(lons[lons != np.inf])

# Query IBTraCS data based on bounding box
print('Number of records before checking for out-of-bounds tracks:',len(df))
df = df.loc[(df['LAT'] >= min_lat) & (df['LAT'] <= max_lat) & (df['LON'] >= min_lon) & (df['LON'] <= max_lon)]
print('Number of records after checking for out-of-bounds tracks:',len(df))

Number of records before checking for out-of-bounds tracks: 55565
Number of records after checking for out-of-bounds tracks: 19062


## Drop Additional Off-Disc Samples 

Now that we've limited the extent a bit, let's drill down and make sure none of the points are off of the full-disc. First, we find the point on the image that corresponds to the latitude/longitude of the storm. Since the `lats` and `lons` arrays have values of `NaN` where the points are off the disc, we can check if that point in the `lat` or `lon` array is missing. We only need to check one array, since it's a meshgrid, se we'll check `lons`. And rather than just checking if that one point is `NaN`, let's check if any point within a window of size `check_size` is `NaN`. This will avoid any points that are just barely sitting on the edge of the disc.

In [6]:
# Create empty list
drop_inds = []

# Reset indices of dataframe
df = df.reset_index()

for i, row in df[['LAT','LON']].iterrows():
    
    # Cast latitude and longitude to float
    track_lat = float(row["LAT"])
    track_lon = float(row["LON"])

    # Convert lon/lat to x/y
    track_x,track_y = p(track_lon,track_lat)
    if track_x==np.inf or track_y==np.inf or track_x==np.nan or track_y==np.nan:
        drop_inds.append(i)

    # Get the closest point to the IBTrACS data
    x_ind = np.nanargmin(abs(x-track_x))
    y_ind = np.nanargmin(abs(y-track_y))

    # Check that none of 50 points on any side of the storm are off of the disc
    check_size = 50
    off_disc = np.isnan(lons[y_ind-check_size:y_ind+check_size,x_ind-check_size:x_ind+check_size]).any()
    off_disc += np.isinf(lons[y_ind-check_size:y_ind+check_size,x_ind-check_size:x_ind+check_size]).any()

    # If the points are off the disc, append the dataframe index to drop after looping
    if off_disc:
        drop_inds.append(i)

# Drop any indices that fell off the disc
print('Number of records before checking for out-of-bounds tracks:',len(df))
df = df.drop(df.index[drop_inds])
print('Number of records after checking for out-of-bounds tracks:',len(df))

Number of records before checking for out-of-bounds tracks: 19062
Number of records after checking for out-of-bounds tracks: 18396


## Save the Output

So now, we've handled the issues of the imager extent and the IBTrACS extent, both spatially and temporally. Let's save the output now so we can use it to make training data.

In [7]:
df.to_csv('../data/ibtracs/ibtracs_goes_east.csv')